# EV Charging Load Forecasting: Improved Model (GRU)

## 1. Objective
Improve upon the baseline LSTM model by incorporating:
- **Longer Sequence Length:** 168 hours (1 week) to capture weekly seasonality.
- **Feature Engineering:** Lag features (24h, 168h) and calendar features.
- **Model Architecture:** GRU (Gated Recurrent Unit) which is often more efficient and effective for this scale of data.

## 2. Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os

# Configuration
SEQ_LENGTH = 168  # Look back 1 week
BATCH_SIZE = 64
EPOCHS = 100
LEARNING_RATE = 0.001
HIDDEN_SIZE = 128
NUM_LAYERS = 2
DROPOUT = 0.2

# Paths
DATA_DIR = '../data'
FILE_HOURLY = os.path.join(DATA_DIR, 'Dataset_2_Hourly_EV_per_user.csv')
FILE_TRAFFIC = os.path.join(DATA_DIR, 'Dataset 6_Local traffic distribution.csv')
FILE_WEATHER = os.path.join(DATA_DIR, 'Norway_Trondheim_ExactLoc_Weather.csv')

## 3. Advanced Data Preparation
We add Lag features and Day-of-Week encodings.

In [ ]:
def prepare_dataset_improved():
    # 1. Load Hourly Load
    df_hourly = pd.read_csv(FILE_HOURLY, sep=';', decimal=',')
    df_hourly['datetime'] = pd.to_datetime(df_hourly['date_from'], format='%d.%m.%Y %H:%M')
    load_cols = [c for c in df_hourly.columns if 'kW' in c]
    df_hourly[load_cols] = df_hourly[load_cols].fillna(0)
    df_hourly['total_load'] = df_hourly[load_cols].sum(axis=1)
    df_main = df_hourly[['datetime', 'total_load']]

    # 2. Load Traffic
    df_traffic = pd.read_csv(FILE_TRAFFIC, sep=';', decimal=',')
    df_traffic['datetime'] = pd.to_datetime(df_traffic['Date_from'], format='%d.%m.%Y %H:%M')
    traffic_cols = df_traffic.select_dtypes(include=[np.number]).columns
    df_traffic['total_traffic'] = df_traffic[traffic_cols].sum(axis=1)
    df_main = pd.merge(df_main, df_traffic[['datetime', 'total_traffic']], on='datetime', how='left').fillna(0)

    # 3. Load Weather
    df_weather = pd.read_csv(FILE_WEATHER)
    df_weather['date'] = pd.to_datetime(df_weather['datetime']).dt.date
    df_main['date'] = df_main['datetime'].dt.date
    df_main = pd.merge(df_main, df_weather[['date', 'temp', 'solar_rad']], on='date', how='left').fillna(0)
    
    # 4. Feature Engineering
    # Cyclical Time
    df_main['hour_sin'] = np.sin(2 * np.pi * df_main['datetime'].dt.hour / 24)
    df_main['hour_cos'] = np.cos(2 * np.pi * df_main['datetime'].dt.hour / 24)
    df_main['day_sin'] = np.sin(2 * np.pi * df_main['datetime'].dt.dayofweek / 7)
    df_main['day_cos'] = np.cos(2 * np.pi * df_main['datetime'].dt.dayofweek / 7)
    
    # Lags (Shift the target variable)
    # Be careful: We can use known past values as features.
    df_main['lag_24'] = df_main['total_load'].shift(24)
    df_main['lag_168'] = df_main['total_load'].shift(168)
    
    # Drop NaNs created by lags
    df_main = df_main.dropna().reset_index(drop=True)
    
    features = ['total_load', 'total_traffic', 'temp', 'solar_rad', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'lag_24', 'lag_168']
    return df_main, features

df, features = prepare_dataset_improved()
print(f'Dataset Shape: {df.shape}')
print(f'Features: {features}')

In [ ]:
# Split and Scale
train_size = int(len(df) * 0.8)
train_df = df.iloc[:train_size]
test_df = df.iloc[train_size:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_df[features])
test_scaled = scaler.transform(test_df[features])

def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length][0] # Target: total_load
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

X_train, y_train = create_sequences(train_scaled, SEQ_LENGTH)
X_test, y_test = create_sequences(test_scaled, SEQ_LENGTH)

train_loader = DataLoader(TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train)), 
                          shuffle=True, batch_size=BATCH_SIZE)
test_loader = DataLoader(TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test)), 
                         shuffle=False, batch_size=BATCH_SIZE)

## 4. GRU Model Definition
Using GRU with Dropout for better regularization.

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size=1, dropout=0.2):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return out

model = GRUModel(input_size=len(features), hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS, dropout=DROPOUT)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Training
losses = []
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    losses.append(avg_loss)
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {avg_loss:.4f}')

In [ ]:
# Evaluation
model.eval()
preds, actuals = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        preds.extend(model(X_batch).squeeze().numpy())
        actuals.extend(y_batch.numpy())

def inverse_target(scaled_target, scaler, features):
    dummy = np.zeros((len(scaled_target), len(features)))
    dummy[:, 0] = scaled_target
    return scaler.inverse_transform(dummy)[:, 0]

y_pred_real = inverse_target(preds, scaler, features)
y_test_real = inverse_target(actuals, scaler, features)

mae = mean_absolute_error(y_test_real, y_pred_real)
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))
r2 = r2_score(y_test_real, y_pred_real)

print(f'Improved MAE: {mae:.4f}')
print(f'Improved RMSE: {rmse:.4f}')
print(f'Improved R2: {r2:.4f}')